In [87]:
from langchain_experimental.agents import create_csv_agent
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
import os
load_dotenv()
import pandas as pd
from langchain_community.document_loaders import JSONLoader
import requests

In [2]:
llm = AzureChatOpenAI(openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
        azure_deployment=os.environ.get("AZURE_OPENAI_DEPLOYMENT", "gpt4chat"),
        azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
        api_key=os.environ.get("AZURE_OPENAI_KEY"))

In [5]:

def metadata_func(record: str, metadata: dict) -> dict:
    lines = record.split('\n')
    locality_line = lines[10]
    price_range_line = lines[12]
    locality = locality_line.split(': ')[1]
    price_range = price_range_line.split(': ')[1]
    metadata["location"] = locality
    metadata["price_range"] = price_range

    return metadata

# Instantiate the JSONLoader with the metadata_func
jq_schema = '.parser[] | to_entries | map("\(.key): \(.value)") | join("\n")'
loader = JSONLoader(
    jq_schema=jq_schema,
    file_path='data.json',
    metadata_func=metadata_func,
)

# Load the JSON file and extract metadata
documents = loader.load()

# Print the metadata of the first document
print(documents[0].metadata)

{'source': '/Users/siddartha/Desktop/github/Traversaal_ai/data.json', 'seq_num': 1, 'location': 'Istanbul', 'price_range': 'low'}


In [14]:
print(type(documents[0]))

<class 'langchain_core.documents.base.Document'>


In [21]:
text = [i.page_content for i in documents]

In [34]:
from langchain_openai import AzureOpenAIEmbeddings
embeddings = AzureOpenAIEmbeddings(
    openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
    azure_deployment=os.environ.get("embeddings","embed"),
    azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT", "https://gpt-4-trails.openai.azure.com/"),
    api_key=os.environ.get("AZURE_OPENAI_KEY")
)

In [57]:
from langchain_openai import OpenAIEmbeddings
import os

In [58]:
from langchain.vectorstores import FAISS
def get_vectorstore(text_chunks):
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents=text_chunks, embedding=embeddings)
    return vectorstore

In [59]:
vector = get_vectorstore(documents)

In [210]:
def api_call(text):
  url = "https://api-ares.traversaal.ai/live/predict"

  payload = { "query": [text]}
  headers = {
    "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
    "content-type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)

  # here we will use the llm to summarize the response received from the ares api
  response_data = response.json()
  #print(response_data)
  try:
    response_text = response_data['data']['response_text']
    web_urls = response_data['data']['web_url']
    # Continue processing the data...
  except KeyError:
    print("Error: Unexpected response from the API. Please try again or contact the api owner.")
    # Optionally, you can log the error or perform other error handling actions.
  

  if len(response_text) > 10000:
    response_text = response_text[:8000]
    prompt = f"Summarize the following text in 500-100 0 words and jsut summarize what you see and do not add anythhing else: {response_text}"
    summary = llm.invoke(prompt)
    print(summary)
  else:
    summary = response_text

  result = "{} My list is: {}".format(response_text, web_urls)

# Convert the result to a string
  result_str = str(result)

  return result_str

In [211]:
api_call("tacos in paris")

"The French tacos is a popular fast food dish in France that originated in the Rhône-Alpes region around the turn of the twenty-first century. It is a sandwich made with a flour tortilla, condiments, meat (usually halal), French fries, and cheese sauce. The French tacos is a fusion of panini, kebab, and burrito, and it has gained popularity in recent years, with chain restaurants dedicated to serving French tacos opening across the country. The exact origins of the French tacos are debated, but it is commonly associated with snack bars owned by individuals of North African descent in the suburbs of Lyon. The French tacos has become a symbol of suburban pride and a source of economic opportunity for many entrepreneurs. While some Mexican restaurateurs in France see it as cultural appropriation, the French tacos has found a place in French cuisine and is enjoyed by many. My list is: ['https://www.pariseater.com/restaurants/mexican-restaurants-in-paris/', 'https://www.newyorker.com/magazi

In [208]:
import json
def ares_api_call(query):
    url = "https://api.traversaal.ai/live/predict"
    payload = {"query": [query]}
    headers = {
        "x-api-key": "aares_1790bf3106074d9bb5919475f11292bb774f61ca24ac483bb956b37637dee1cd",
        "content-type": "application/json"
    }
    response = requests.post(url, json=payload, headers=headers)

    return response

In [209]:
ares_api_call("What is the weather in New York?")

<Response [405]>

In [223]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationSummaryMemory

template = """

context:- I have low budget what is the best hotel in Instanbul?
anser:- The other hotels in instanbul are costly and are not in your budget. so the best hotel in instanbul for you is hotel is xyz."

Don’t give information not mentioned in the CONTEXT INFORMATION. 
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations.
{context} 
Question: {question}
Answer:"""

prompt = PromptTemplate(template=template, input_variables=["context","question"])

chain_type_kwargs = {"prompt": prompt}
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)

In [224]:
question = "give me deatils about expensive hotels in paris?"

final_response = chain.run(question)
print(final_response)

Here are some details about expensive hotels in Paris:

1. Grand Hotel du Palais Royal:
   - Description: A 5-star hotel located in the heart of Paris. It offers a unique experience with its privileged location near the Palais Royal, Louvre, and Jardin des Tuileries.
   - Review: A guest mentioned that it was a wonderful stay in a beautiful hotel in the most beautiful city in the world. They praised the great staff, great location, comfortable rooms, and the kindness, courtesy, and accuracy of the hotel.
   - Rating: 5 out of 5
   - Review Count: 1502
   - Price Range: High
   - Address: 4 Rue de Valois, Paris, France
   - [More Information](https://www.tripadvisor.com/Hotel_Review-g187147-d617625-Reviews-or30-Grand_Hotel_du_Palais_Royal-Paris_Ile_de_France.html)

2. Hotel La Comtesse:
   - Description: No description available.
   - Review: A guest mentioned that the staff was wonderful with amazing customer service. They loved the comfortable room and the view of the Eiffel Tower. Th

In [141]:
api_response = api_call(question)
prompt = """Please write the response to the user query: using the final_response and api_resource and make sure you are
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations. Do not add any information that is not mentioned in the context.
and make sure the answer is up to the point and not too long.

question: when did sachin hit his 100th century?
final_response: I can you assist you with hotel's or travels or food but cannot help other than that..

"""
response = llm.invoke(prompt+question+final_response + api_response)

print(response.content)

The Grand Hotel du Palais Royal, Le Meurice, Plaza Athénée, Shangri-La Paris, Mandarin Oriental, The Ritz Hotel, Four Seasons Hotel, Peninsula Paris, and Rosewood Hotel are some of the most expensive hotels in Paris. These hotels offer luxury accommodations, with prices that can reach up to $36,000 per night. They are located in prestigious areas of the city, near attractions such as the Louvre Museum, the Place de la Concorde, and the Eiffel Tower. The hotels are praised for their luxurious decor, spacious rooms, and excellent service. However, they have also received mixed reviews from guests. More details can be found on their respective websites.


In [142]:
print(llm.invoke("Can you help me get the hotel links?"))

content="Sure, I'd be happy to help. However, I'll need a bit more information. Could you please tell me the names of the hotels and their locations?"


In [144]:
from langchain.chains import ConversationChain, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chains.conversational_retrieval.prompts import QA_PROMPT
from langchain.prompts.chat import SystemMessagePromptTemplate,HumanMessagePromptTemplate
from langchain.prompts import PromptTemplate, ChatPromptTemplate

promptTemplate = """
context:- I have low budget what is the best hotel in Instanbul?
anser:- The other hotels in instanbul are costly and are not in your budget. so the best hotel in instanbul for you is hotel is xyz."

Don’t give information not mentioned in the CONTEXT INFORMATION. 
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations.
{context} 
Question: {question}
Answer:"""

prompt = PromptTemplate(template=promptTemplate, input_variables=["context","question"])

chain_type_kwargs = {"prompt": prompt}

history = []
memory = ConversationSummaryMemory(
    memory_key='chat_history', return_messages=True, output_key='answer', llm=llm
)

messages = [
    SystemMessagePromptTemplate.from_template(promptTemplate),
    HumanMessagePromptTemplate.from_template("{question}")
]

qa_prompt = ChatPromptTemplate.from_messages(messages)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm, vector.as_retriever(), memory=memory, get_chat_history=lambda h: h,
    combine_docs_chain_kwargs={"prompt": qa_prompt}
)

chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)


In [180]:
def chat_response(question):
    answer_for_message = ""
    answer = qa_chain.invoke({"question":question,"chat_history":history})#,return_only_outputs=True)
    answer = answer["answer"]
    prompt = """Please write the response to the user query: using the final_response and api_resource and make sure you are
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations and also provide them with website links. Do not add any information that is not mentioned in the context.
and make sure the answer is up to the point and not too long.

question: when did sachin hit his 100th century?
final_response: I can you assist you with hotel's or travels or food but cannot help other than that..

question: what year did messi won the world cup?
final_response: I can you assist you with hotel's or travels or food but cannot help other than that.

use the above 2 response only when the query is not related to the hotel or travel or food.

"""
    if answer_for_message == "":
        api_answer = api_call(question,answer_for_message)
    else:
        api_answer = api_call(question,answer_for_message.content)
    final_answer = llm.invoke(prompt+question+answer+api_answer)
    answer_for_message = llm.invoke("Summarize as much as you can but make sure you are notloosing any important information."+final_answer.content)
    history.append((question, answer_for_message.content))
    return final_answer.content

In [181]:
print(chat_response("give me deatils about expensive hotels in paris?"))

Based on the factors such as location, amenities, and user reviews, here are some of the high-priced hotels in Paris:

1. [Grand Hotel du Palais Royal](https://www.tripadvisor.com/Hotel_Review-g187147-d617625-Reviews-or20-Grand_Hotel_du_Palais_Royal-Paris_Ile_de_France.html): This 5-star hotel is located in the heart of Paris. It is close to the Palais Royal, the Louvre, and the Jardin des Tuileries. The hotel has a high price range and a rating value of 5 based on 1502 reviews.

2. [Le Meurice](https://www.dorchestercollection.com/en/paris/le-meurice/): This luxury hotel offers rooms decorated in the style of King Louis XVI, and its Presidential suite can cost up to $8,800 per night.

3. [Plaza Athénée](https://www.dorchestercollection.com/en/paris/hotel-plaza-athenee/): This hotel has spacious rooms with marble bathrooms and beautiful dressing rooms. The Eiffel Suite can cost up to $15,000 per night.

4. [Shangri-La Paris](https://www.shangri-la.com/paris/shangrila/): This hotel offe

In [182]:
print(chat_response("Can you give me the links of websites for the above hotels?"))

Sure, here are the website links for the hotels mentioned above:

1. Radisson Blu Hotel Istanbul Ottomare: [https://www.tripadvisor.com/Hotel_Review-g293974-d9459106-Reviews-or30-Radisson_Blu_Hotel_Istanbul_Ottomare-Istanbul.html](https://www.tripadvisor.com/Hotel_Review-g293974-d9459106-Reviews-or30-Radisson_Blu_Hotel_Istanbul_Ottomare-Istanbul.html)

2. Park Grand London Hyde Park: [https://www.tripadvisor.com/Hotel_Review-g186338-d3164384-Reviews-or10-Park_Grand_London_Hyde_Park-London_England.html](https://www.tripadvisor.com/Hotel_Review-g186338-d3164384-Reviews-or10-Park_Grand_London_Hyde_Park-London_England.html)

3. Wyndham Grand Istanbul Kalamis Marina Hotel: [https://www.tripadvisor.com/Hotel_Review-g293974-d3588557-Reviews-or30-Wyndham_Grand_Istanbul_Kalamis_Marina_Hotel-Istanbul.html](https://www.tripadvisor.com/Hotel_Review-g293974-d3588557-Reviews-or30-Wyndham_Grand_Istanbul_Kalamis_Marina_Hotel-Istanbul.html)

4. Royal National Hotel: [https://www.tripadvisor.com/Hotel_R

In [212]:
import os
from typing import List, Tuple
import json
import uvicorn
from fastapi import FastAPI
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.callbacks import FinalStreamingStdOutCallbackHandler
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.pydantic_v1 import BaseModel, Field
from langchain.schema.messages import AIMessage, HumanMessage
from langchain.tools.render import format_tool_to_openai_function
from langchain_community.utilities.google_serper import GoogleSerperAPIWrapper
from langchain_core.runnables import ConfigurableField
from langchain_core.tools import Tool
from langserve import add_routes
from langchain.prompts import PromptTemplate
import requests
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Qdrant
from langchain.chains import RetrievalQA
from langchain.agents import Tool, Agent, AgentType
from langchain.agents import AgentExecutor
from langchain_core.tools import Tool
from langchain_openai import ChatOpenAI
from langchain_openai import AzureChatOpenAI
from langchain_community.document_loaders import JSONLoader

In [220]:
llm_1 = ChatOpenAI(temperature=0.2,
                 model="gpt-3.5-turbo-0125",
                 streaming=True,
                 callbacks=[FinalStreamingStdOutCallbackHandler()]).configurable_fields(
    temperature=ConfigurableField(
        id="llm_temperature",
        name="LLM Temperature",
        description="The temperature of the LLM"))
assistant_system_message = """You are a helpful assistant. \
Use tools (only if necessary) to best answer the users questions."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", assistant_system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# Define the API call function for Ares API

def api_call(text):
  url = "https://api-ares.traversaal.ai/live/predict"

  payload = { "query": [text]}
  headers = {
    "x-api-key": "ares_a0866ad7d71d2e895c5e05dce656704a9e29ad37860912ad6a45a4e3e6c399b5",
    "content-type": "application/json"
  }

  response = requests.post(url, json=payload, headers=headers)

  # here we will use the llm to summarize the response received from the ares api
  response_data = response.json()
  #print(response_data)
  try:
    response_text = response_data['data']['response_text']
    web_urls = response_data['data']['web_url']
    # Continue processing the data...
  except KeyError:
    print("Error: Unexpected response from the API. Please try again or contact the api owner.")
    # Optionally, you can log the error or perform other error handling actions.
  

  if len(response_text) > 10000:
    response_text = response_text[:8000]
    prompt = f"Summarize the following text in 500-100 0 words and jsut summarize what you see and do not add anythhing else: {response_text}"
    summary = llm_1.invoke(prompt)
    print(summary)
  else:
    summary = response_text

  result = "{} My list is: {}".format(response_text, web_urls)

# Convert the result to a string
  result_str = str(result)

  return result_str



# def metadata_func(record: str, metadata: dict) -> dict:
#     lines = record.split('\n')
#     locality_line = lines[10]
#     price_range_line = lines[12]
#     locality = locality_line.split(': ')[1]
#     price_range = price_range_line.split(': ')[1]
#     metadata["location"] = locality
#     metadata["price_range"] = price_range

#     return metadata

# # Instantiate the JSONLoader with the metadata_func
# jq_schema = '.parser[] | to_entries | map("\(.key): \(.value)") | join("\n")'
# loader = JSONLoader(
#     jq_schema=jq_schema,
#     file_path='data.json',
#     metadata_func=metadata_func,
# )

# # Load the JSON file and extract metadata
# documents = loader.load()

    
# from langchain.vectorstores import FAISS
# def get_vectorstore(text_chunks):
#     embeddings = OpenAIEmbeddings()
#     vectorstore = FAISS.from_documents(documents=text_chunks, embedding=embeddings)
#     return vectorstore


# vector = get_vectorstore(documents)



def search(text):
    #llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0)
    vector = vector
    prompt = PromptTemplate(template=template, input_variables=["context","question"])
    chain_type_kwargs = {"prompt": prompt}
    return RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector.as_retriever(),
    chain_type_kwargs=chain_type_kwargs,
)

# Initialize LangChain tools
template = """

context:- I have low budget what is the best hotel in Instanbul?
anser:- The other hotels in instanbul are costly and are not in your budget. so the best hotel in instanbul for you is hotel is xyz."

Don’t give information not mentioned in the CONTEXT INFORMATION. 
The system should take into account various factors such as location, amenities, user reviews, and other relevant criteria to 
generate informative and personalized explanations.
{context} 
Question: {question}
Answer:"""

api_tool = Tool(name="Ares_API",
                func=api_call,
                description="Integration with Traversaal AI Ares API for real-time internet searches."
               )

chain_rag_tool = Tool(name="RAG_Chain",
                      func=search,
                      description="RAG chain for question answering."
                     )


tools = [chain_rag_tool, api_tool]
llm_with_tools = llm.bind(functions=[format_tool_to_openai_function(t) for t in tools])


def _format_chat_history(chat_history: List[Tuple[str, str]]):
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer


agent = (
    {
        "input": lambda x: x["input"],
        "chat_history": lambda x: _format_chat_history(x["chat_history"]),
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)


class AgentInput(BaseModel):
    input: str
    chat_history: List[Tuple[str, str]] = Field(
        ..., extra={"widget": {"type": "chat", "input": "input", "output": "output"}}
    )


agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_types(
    input_type=AgentInput
)

In [221]:
def get_response(user_input):
    response = agent_executor.invoke({"input":user_input, "chat_history": _format_chat_history([])})
    return response

# Define a function to start the conversation with the user
def start_conversation():
    print("Chatbot: Hi there! How can I assist you with your hotel preferences today?")
    while True:
        user_input = input("User: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Chatbot: Goodbye! Have a great day.")
            break
        response = get_response(user_input)
        print("Chatbot:", response['output'])

# Start the conversation
start_conversation()

Chatbot: Hi there! How can I assist you with your hotel preferences today?


> Entering new AgentExecutor chain...

Invoking: `Ares_API` with `high budget hotel in istanbul`


1. The Pera Palace is a high-budget hotel in Istanbul that offers a luxurious and historic experience. It has been the preferred lodging for famous guests such as Ernest Hemingway and Agatha Christie. The hotel features stunning Ottoman architecture, original art, and furnishings. Guests can enjoy a mix of French, Italian, and Turkish cuisine at the Orient Terrace.

2. The Ciragan Palace Kempinski is another high-budget luxury hotel in Istanbul. It is housed inside an opulent palace originally built in the 17th century. The hotel offers unrivaled views of the Bosphorus and features a heated infinity pool, poolside bar, and restaurant. Guests can also savor Ottoman flavors at the Gazebo Lounge restaurant.

3. The Vault Karakoy House Hotel is a unique luxury hotel in Istanbul. It is housed inside a century-old Turk

UnboundLocalError: local variable 'vector' referenced before assignment